In [ ]:
from meteostat import Stations, Daily
from datetime import datetime
import pandas as pd

start = datetime(2015, 8, 12)
end = datetime(2019, 1, 23)

# Coordinates of several major Slovenian cities
locations = {
    'Ljubljana': (46.05, 14.51),
    'Maribor': (46.56, 15.64),
    'Koper': (45.55, 13.73),
    'Celje': (46.23, 15.27)
}

dfs = []

for city, (lat, lon) in locations.items():
    station = Stations().nearby(lat, lon).fetch(1)
    
    if not station.empty:
        station_id = station.index[0]  # The station ID is now in the index
        data = Daily(station_id, start, end).fetch()
        data = data.rename(columns=lambda x: f"{city}_{x}")
        dfs.append(data)
    else:
        print(f"⚠️ No station found near {city}.")

# Combine and compute mean across all cities
if dfs:
    combined = pd.concat(dfs, axis=1)
    aggregated = combined.groupby(level=0, axis=1).mean()
    aggregated.to_csv("slovenia_weather_aggregated.csv")
    print("✅ Saved: slovenia_weather_aggregated.csv")
else:
    print("❌ No dataframes to combine.")


In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("slovenia_weather_aggregated.csv", parse_dates=["time"])
df = df.set_index("time")

# Extract unique feature types from the column names
features = ['prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin', 'tsun', 'wdir', 'wpgt', 'wspd']

# Initialize dictionary to hold averaged columns
avg_data = {}

for feature in features:
    # Get all columns that end with the current feature
    feature_cols = [col for col in df.columns if col.endswith(f"_{feature}")]
    
    if feature_cols:
        # Average while skipping NaNs
        avg_data[feature] = df[feature_cols].mean(axis=1)

# Create DataFrame from the averaged values
df_avg = pd.DataFrame(avg_data, index=df.index)

# Optional: Save to CSV
df_avg.to_csv("slovenia_weather_averaged.csv")


In [1]:
import requests
import pandas as pd
from datetime import datetime

def fetch_open_meteo(city_name, lat, lon, start_date, end_date):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,precipitation,surface_pressure,snowfall,"
        f"shortwave_radiation,winddirection_10m,windgusts_10m,windspeed_10m,"
        f"relative_humidity_2m&timezone=Europe%2FBerlin"
    )
    print(f"Fetching: {city_name}")
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data for {city_name}: {response.text}")

    data = response.json()["hourly"]
    df = pd.DataFrame(data)
    df["datetime"] = pd.to_datetime(df["time"])
    df["city"] = city_name
    return df.drop(columns=["time"])

# Setup
start = "2019-01-24"
end = "2022-01-24"

cities = {
    "Ljubljana": (46.0569, 14.5058),
    "Maribor": (46.5547, 15.6459),
    "Celje": (46.2309, 15.2604),
    "Koper": (45.5481, 13.7302),
    "Novo Mesto": (45.8030, 15.1688)
}

# Fetch and concatenate
dfs = [fetch_open_meteo(city, lat, lon, start, end) for city, (lat, lon) in cities.items()]
all_data = pd.concat(dfs)

# Aggregate to country level (mean)
country_hourly = (
    all_data
    .drop(columns=["city"])
    .groupby("datetime")
    .mean(numeric_only=True)
    .round(4)
    .reset_index()
)


# Save
country_hourly.to_csv("slovenia_hourly_weather_future.csv", index=False)
print("Saved to slovenia_hourly_weather_openmeteo.csv")


Fetching: Ljubljana
Fetching: Maribor
Fetching: Celje
Fetching: Koper
Fetching: Novo Mesto
Saved to slovenia_hourly_weather_openmeteo.csv


In [4]:
df = pd.read_csv('slovenia_hourly_weather_openmeteo.csv')
df = df.round(4)
df.to_csv('slovenia_hourly_weather_openmeteo.csv')